## Import necessary python modules

In [1]:
import os
import napari
import tifffile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import bigfish
import bigfish.plot as plot
import bigfish.stack as stack
import bigfish.detection as detection
import bigfish.multistack as multistack

import tkinter as tk
from tkinter import filedialog
from random import sample 

from copy import deepcopy
from dask.array.image import imread as imr
from bigfish.detection.utils import get_object_radius_pixel
from buildReferenceSpot import buildReferenceSpotFromImages
from runBigfishDetection import getSpotAndClusters, saveSpotsNPZ, reorderZstack

from pathlib import Path
from napari.layers import Image
from magicgui import magicgui, magic_factory, widgets
from magicgui.widgets import Table

## Specify voxel and object size

In [2]:
voxelRadius = (600, 121, 121)
objectRadius = (600, 150, 150)

In [3]:
from scipy.optimize import curve_fit

def bi_exp(x, a, b, c, d):
    return (a * np.exp(-b * x)) + (c * np.exp(-d * x))

def trip_exp(x, a, b, c, d, e, f):
    return ((a * np.exp(-b * x)) + (c * np.exp(-d * x)) + (e * np.exp(-f * x)))

def getBleachCorrected(stackCell, model='bi'):
    axes = tuple([i for i in range(len(stackCell.shape))])
    I_mean = np.mean(stackCell, axis=axes[1:])
    timePoints = np.arange(stackCell.shape[0])
    
    if model=='bi':
        coeffsExp, _ = curve_fit(bi_exp, timePoints, I_mean, maxfev=50000)
        f_ = np.vectorize(bi_exp)(timePoints, *coeffsExp)
    elif model=='tri':
        coeffsExp, _ = curve_fit(trip_exp, timePoints, I_mean, maxfev=50000)
        f_ = np.vectorize(trip_exp)(timePoints, *coeffsExp)
    
    
    f = f_ / np.max(f_)
    f = f.reshape(-1, 1, 1, 1)
    imagesCorrected = (stackCell / f).astype(np.uint16)

    # calculate r squared
    residuals = I_mean - f_
    ss_res = np.sum(residuals ** 2)
    ss_tot = np.sum((I_mean - np.mean(I_mean)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    r_squared_exp = np.array(r_squared)
    return imagesCorrected, r_squared_exp, I_mean

def getImagesAndSpotList(sequenceCell, selectedThreshold, voxelRadius, objectRadius, sampling=10):
    images=[]    
    spots_list=[]
    MaxTimePoint = sequenceCell.shape[0]

    spot_radius_px = detection.get_object_radius_pixel(
        voxel_size_nm=voxelRadius, 
        object_radius_nm=objectRadius, 
        ndim=3)

    for t in range(1,MaxTimePoint,sampling):
        rna = np.array(sequenceCell[t])
        images.append(rna)

        # LoG filter
        rna_log = stack.log_filter(rna, sigma=spot_radius_px)

        # local maximum detection
        mask = detection.local_maximum_detection(rna_log, min_distance=spot_radius_px)

        # thresholding
        threshold = detection.automated_threshold_setting(rna_log, mask)
        spots_, _ = detection.spots_thresholding(rna_log, mask, float(selectedThreshold))
        spots_list.append(spots_)
    n=len(images)
    print("Total number of images : "+str(n))
    return images, spots_list, n

def choose_home_folder():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    file_path = filedialog.askdirectory(initialdir= "/", title='Please select a directory')  # Open file dialog

    root.destroy()  # Close the tkinter window
    return file_path

In [4]:
midentifier = 'cell_'

In [5]:

baseFolder = choose_home_folder() #homeFolder+nameKey+imsQ
thresholdFile = pd.read_csv(baseFolder+'/thresholds.csv', index_col=0)


print("Chosen home folder:", baseFolder) # folder containg folder of movies
sessionNames = [os.path.join(baseFolder+'/'+i) for i in os.listdir(baseFolder) if midentifier in i and os.path.isdir(os.path.join(baseFolder,i))]

Chosen home folder: C:/Users/uid-1204/Desktop/test2_bigFISHLIVE/Hela_h9_h2_k11_mcpsg_1hrbasal_14hr_TNF_TSA_exp2_2_F06


In [8]:
thresholdFile

,cell,threshold,Tx_Sites
r0,cell_1,600,0
r1,cell_10,0,0
r2,cell_11,0,0
r3,cell_12,600,0
r4,cell_13,0,0
r5,cell_14,620,0
r6,cell_16,598,0
r7,cell_17,0,0
r8,cell_18,598,0
r9,cell_2,830,0


In [9]:
from tkinter import *
 
# Create the root window
root = Tk()
root.geometry('180x200')
 
# Create a listbox
listbox = Listbox(root, width=90, height=10, selectmode=MULTIPLE)
 
# Inserting the listbox items
for i in range(len(sessionNames)):
    listbox.insert(i+1, sessionNames[i].split('/')[-1])

# Function for printing the
# selected listbox value(s)
def selected_item():
    global chosenCell
    # Traverse the tuple returned by
    # curselection method and print
    # corresponding value(s) in the listbox
    chosenCell = listbox.get(listbox.curselection())
    root.destroy()
    
# Create a button widget and
# map the command parameter to
# selected_item function
btn = Button(root, text='Choose', command=selected_item)
 
# Placing the button and listbox
btn.pack(side='bottom')
listbox.pack()
 
root.mainloop()

In [10]:
chosenCell

'cell_14'

## Specify Input and Output folders, identifiers

In [11]:

referenceSpot_Alpha=0.7
referenceSpot_gamma=3

reorderStack = False
opbCorrection = False
fieldNumber = thresholdFile[thresholdFile['cell']==chosenCell].index[0]
print('cell number :', thresholdFile['cell'][fieldNumber])
cellNumber = thresholdFile['cell'][fieldNumber]
for sessionIndex in range(len(sessionNames)):
    if cellNumber in sessionNames[sessionIndex]:
        print('found :', sessionIndex)
        break
path_input = os.path.join(sessionNames[sessionIndex])
pathToTimeFramesCell = path_input+'/*.tif'

sequenceCell = imr(pathToTimeFramesCell)
mipSequenceCell = np.max(sequenceCell, axis=1)
MaxTimePoint = sequenceCell.shape[0]
selectedThreshold = float(thresholdFile['threshold'][fieldNumber])
print('selected threshold : ', selectedThreshold)



if selectedThreshold!=0:
    if opbCorrection==True:
        sequenceCellInput, r_bi, I_mean = getBleachCorrected(sequenceCell, model='bi')
    else:
        sequenceCellInput = sequenceCell
    ## Get a list of spots detected using the threshold specified in previous step
    images, spots_list, no_frames = getImagesAndSpotList(sequenceCellInput, selectedThreshold, voxelRadius, objectRadius, sampling=6)


    ## Build reference spot
    reference_spot, spots_found, max_intensity = buildReferenceSpotFromImages(images, spots_list,                                                                          alpha=referenceSpot_Alpha, 
                                                                      gamma=referenceSpot_gamma, voxelSize=voxelRadius, objectSize=objectRadius)

cell number : cell_14
found : 5
selected threshold :  620.0
Total number of images : 150
image list found!
Found 4032
Found 4032 spots, max intensity = 3201


In [12]:
VOXEL_RADIUS = (0,0,0)
OBJECT_RADIUS = (0,0,0)

DEFAULT_CHOICES=['None']
SET_TABLE = 0
CELL_CHOICE = 0
DEBUG = False

sampleNumber = 5
sampleFrames = sample(range(MaxTimePoint),sampleNumber)
stackCell = sequenceCell[sampleFrames]

@magicgui(main_window=False,
          call_button='Set voxel parameters',
          voxel_radius_z = {"widget_type": "IntSlider", 'max': 1000},
          voxel_radius_xy = {"widget_type": "IntSlider", 'max': 1000},
          object_radius_z = {"widget_type": "IntSlider", 'max': 1000},
          object_radius_xy = {"widget_type": "IntSlider", 'max': 1000})

def setVoxelandObjectParameters(voxel_radius_z=600,
                                voxel_radius_xy = 121,
                                object_radius_z=600,
                                object_radius_xy=150,
    ):

    voxelRadius = (voxel_radius_z, voxel_radius_xy, voxel_radius_xy)
    objectRadius = (object_radius_z, object_radius_xy, object_radius_xy)
    return voxelRadius, objectRadius
@setVoxelandObjectParameters.called.connect
def on_set_vb_parameters(value: list):
    global VOXEL_RADIUS
    global OBJECT_RADIUS
    VOXEL_RADIUS = value[0]
    OBJECT_RADIUS = value[1]
    if DEBUG==True: 
        print('voxel ', VOXEL_RADIUS)
        print('object', OBJECT_RADIUS)
@magicgui(main_window=False,
        call_button="Verify threshold",
        threshold = {"widget_type": "IntSlider", 'max': 6000},
        beta = {"widget_type": "FloatSlider", 'max': 5},
        gamma = {"widget_type": "IntSlider", 'max': 30},
        number_of_min_spots = {"widget_type": "IntSlider", 'max': 10},
        radius_of_cluster_nm = {"widget_type": "IntSlider", 'max': 1000},
         )

def recordThreshold(img_layer: "napari.layers.Image", threshold=selectedThreshold, 
                    beta=1.2, 
                    gamma=3, 
                    number_of_min_spots=3,
                    radius_of_cluster_nm=600
                   ):
    image_data = img_layer.data
    stackCellSample = image_data#image_data[::samples,:,:,:]


    return stackCellSample, threshold, beta, gamma, number_of_min_spots, radius_of_cluster_nm
@recordThreshold.called.connect
def on_verify_threshold(value: str):
    print('calling verify threshold')
    stackCellSample = value[0]
    threshold = value[1]
    spots_current_list = []
    bEta = value[2]
    gAmma = value[3]
    numberSpotmin = value[4]
    radiusCluster_nm  = value[5]

    spotsFrame, clustersFrames, ThresholdFrames = getSpotAndClusters(stackCellSample, 
                                                             reference_spot, 
                                                             cellnumber=cellNumber, 
                                                             startTime=0,
                                                             stopTime=len(stackCellSample), 
                                                             thresholdManual=threshold, 
                                                             beta=bEta, 
                                                             gamma=gAmma,
                                                             numberOfSpots=numberSpotmin,
                                                             radiusCluster=radiusCluster_nm, 
                                                             voxelSize=voxelRadius, 
                                                             objectSize=objectRadius,
                                                             reorder=reorderStack,
                                                             extensionMov='.tif',
                                                             showProgress=False)
    
    spotList = formatSpotsShape(spotsFrame)

    bigfish_Spots = viewer.add_points(
        spotList,
        face_color='#00000000',
        size=8,
        edge_width=0.3,
        edge_width_is_relative=False,
        edge_color='white',
        name = 'bigFish Detected Spots '+str(threshold)
        )
    
    clusterList = formatSpotsShape(clustersFrames)[:,:4]

    bigfish_Clusters = viewer.add_points(
        clusterList,
        face_color='#00000000',
        size=8,
        edge_width=0.4,
        edge_width_is_relative=False,
        edge_color='red',
        name = 'bigFish Clusters '+str(threshold)
        )
    
    
@magicgui(main_window=False,
         call_button="View Frames",
         )
def viewSpecificFrames(frame_number = 1):
    sampleFrames = np.array([frame_number-1,frame_number,frame_number+1])
    stackCell = sequenceCell[sampleFrames]
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    image_layer = viewer.add_image(stackCell,
                             colormap='green',
                                name='stackCell')
    
@magicgui(main_window=False,
         call_button="Resample",
         )
def resampleMovie():
    sampleNumber = 5
    sampleFrames = sample(range(MaxTimePoint),sampleNumber)
    stackCell = sequenceCell[sampleFrames]
    viewer.layers.select_all()
    viewer.layers.remove_selected()
    image_layer = viewer.add_image(stackCell,
                             colormap='green',
                                name='stackCell')



def formatSpotsShape(spotList):
    spotArray = np.hstack([np.ones((len(spotList[0]),1))*0, spotList[0]])
    for i in range(1,len(spotList)):
        spotArray = np.vstack([spotArray,np.hstack([np.ones((len(spotList[i]),1))*i, spotList[i]])])
    return spotArray 


container = widgets.Container(
    widgets=[setVoxelandObjectParameters,
             recordThreshold, 
             resampleMovie,
            viewSpecificFrames], layout="vertical", labels=False
)

container.native.setMinimumWidth(500)

viewer = napari.Viewer()

image_layer = viewer.add_image(
        stackCell, colormap='green' #maxImageCell
        )
viewer.window.add_dock_widget(container, area="right", name = 'threshold estimation')

calling verify threshold
done!
calling verify threshold
done!
calling verify threshold
done!
calling verify threshold
done!
